Valentine Shilov
### Алгоритмы интеллектуальной обработки больших объемов данных
## Домашнее задание №3 - Дерево решений


**Общая информация**

**Срок сдачи:** до 30 апреля 2018, 06:00   
**Штраф за опоздание:** -2 балла после 06:00 30 апреля, -4 балла после 06:00 7 мая, -6 баллов после 06:00 14 мая, -8 баллов после 06:00 21 мая

При отправлении ДЗ указывайте фамилию в названии файла   


Присылать ДЗ необходимо в виде ссылки на свой github репозиторий в slack @alkhamush


Используйте данный Ipython Notebook при оформлении домашнего задания.

###### Задание 1 (3 балла)
Разберитесь в коде MyDecisionTreeClassifier, который уже частично реализован. Допишите код там, где написано "Ваш код". Ваша реализация дерева должна работать по точности не хуже DecisionTreeClassifier из sklearn. Точность проверяется на wine и Speed Dating Data.

###### Задание 2 (3 балла)
Добиться скорости работы на fit не медленнее чем в 10 раз sklearn на данных wine и Speed Dating Data. 
Для этого используем numpy.

###### Задание 3 (2 балла)
Добавьте функционал, который определяет значения feature importance. Выведите 10 главных фичей под пунктом Задание 4 (уже написано ниже) для MyDecisionTreeClassifier и DecisionTreeClassifier так, чтобы сразу были видны выводы и по MyDecisionTreeClassifier, и по DecisionTreeClassifier. Используем данные Speed Dating Data.

###### Задание 4 (2 балла)
С помощью GridSearchCV или RandomSearchCV подберите наиболее оптимальные параметры для случайного леса (Выберете 2-3 параметра). Используем данные Speed Dating Data. Задание реализуйте под пунктом Задание 5 (уже написано ниже)


**Штрафные баллы:**

1. Невыполнение PEP8 -1 балл
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw3.ipynb) -1 балл
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -1 балл
4. При оформлении ДЗ нужно пользоваться данным файлом в качестве шаблона. Не нужно удалять и видоизменять написанный код и текст. В противном случае -1 балл

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.datasets import load_wine
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import KFold, train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier

%matplotlib inline
%load_ext pycodestyle_magic

In [2]:
#%%pycodestyle


class MyDecisionTreeClassifier:
    NON_LEAF_TYPE = 0
    LEAF_TYPE = 1

    def __init__(self, min_samples_split=2, max_depth=None, sufficient_share=1.0, criterion='gini', max_features=None):
        self.tree = dict()
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.sufficient_share = sufficient_share
        self.num_class = -1
        self.feature_importances_ = None
        if criterion == 'gini':
            self.G_function = self.__gini
        elif criterion == 'entropy':
            self.G_function = self.__entropy
        elif criterion == 'misclass':
            self.G_function = self.__misclass
        else:
            print('invalid criterion name')
            raise

        if max_features == 'sqrt':
            self.get_feature_ids = self.__get_feature_ids_sqrt
        elif max_features == 'log2':
            self.get_feature_ids = self.__get_feature_ids_log2
        elif max_features == None:
            self.get_feature_ids = self.__get_feature_ids_N
        else:
            print('invalid max_features name')
            raise

    def __gini(self, l_c, l_s, r_c, r_s):
        l_s = l_s.astype('float')
        r_s = r_s.astype('float')
        return 1 - np.sum((l_c ** 2) / (l_s * (l_s + r_s)) +\
            (np.square(r_c)) / (r_s * (l_s + r_s)), axis=1)
    
    def __entropy(self, l_c, l_s, r_c, r_s):
        return -np.sum(l_c / l_s * np.log2(l_c / l_s), axis=1)-\
            np.sum(r_c/r_s*np.log2(r_c/r_s), axis=1)

    def __misclass(self, l_c, l_s, r_c, r_s):
        return 1 - np.max(l_c / (l_s + r_s), axis=1) -\
            np.max(r_c / (l_s + r_s), axis=1)
    
    def __get_feature_ids_sqrt(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return np.array(feature_ids[:np.int(np.sqrt(n_feature))])
        
    def __get_feature_ids_log2(self, n_feature):
        feature_ids = range(n_feature)
        np.random.shuffle(feature_ids)
        return np.array(feature_ids[:np.int(np.log2(n_feature))])

    def __get_feature_ids_N(self, n_feature):
        return np.arange(n_feature)
    
    def __sort_samples(self, x, y):
        sorted_idx = x.argsort()
        return x[sorted_idx], y[sorted_idx]

    def __div_samples(self, x, y, feature_id, threshold):
        left_mask = x[:, feature_id] > threshold
        right_mask = ~left_mask
        return x[left_mask], x[right_mask], y[left_mask], y[right_mask]

    def __find_threshold(self, x, y):
        #sort
        x_sorted, y_sorted=self.__sort_samples(x,y)
        num_class=self.num_class
        c = self.min_samples_split // 2
        c = np.int(self.min_samples_split / 2 - 1)
        
        #cut middle 
        if(c != 0 ):
            y_middle = y_sorted[c:-c] 
        else:
            y_middle = y_sorted
      
        
        r_delim_ind = np.where(y_middle[1:] != y_middle[:-1])[0] + c + 1
        if len(r_delim_ind) == 0:
            return np.inf, None
        
        eq_element_count = -np.append([c], r_delim_ind[:-1]) + r_delim_ind 
        delim_oht = np.zeros((r_delim_ind.shape[0], num_class))
        delim_oht[np.arange(r_delim_ind.shape[0]), y_sorted[r_delim_ind - 1]] = 1

        class_increments = eq_element_count.reshape(-1, 1) * delim_oht
        class_increments[0] = class_increments[0] + np.bincount(y_sorted[:c], minlength=num_class)
  
        l_class_count = np.cumsum(class_increments, axis=0)
        r_class_count = np.bincount(y_sorted, minlength=num_class) - l_class_count
    
        
        l_count = r_delim_ind.reshape(l_class_count.shape[0], 1)
        r_count = y_sorted.shape[0] - l_count
        
        
        gs = self.G_function(l_class_count, l_count, r_class_count, r_count)
        min_index = np.argmin(gs)
        
        delim_element_index = l_count[min_index][0]
        
        return gs[min_index], (x_sorted[delim_element_index] + x_sorted[delim_element_index - 1]) / 2

    def __fit_node(self, x, y, node_id, depth):
        #create node if not the end
        if  y.size < self.min_samples_split  or \
            (self.max_depth is not None and self.max_depth <= depth) or \
            self.sufficient_share <= np.bincount(y).argmax() / y.size:
            self.tree[node_id] = (self.LEAF_TYPE, np.bincount(y).argmax(), np.bincount(y).astype(float) / y.size)
        #find thresholds
        fids = self.get_feature_ids(x.shape[1])
        thresholds = np.array([self.__find_threshold(x[:, i], y) for i in fids])
        
        
        best_threshold_id = thresholds[:, 0].argmin()
        best_threshold = thresholds[best_threshold_id, 1]
        
        if best_threshold is None:
            #create last leaf
            self.tree[node_id] = (self.LEAF_TYPE, np.bincount(y).argmax(), np.bincount(y).astype(float) / y.size)
            return
       
        x_l, x_r, y_l, y_r = self.__div_samples(x, y, best_threshold_id, best_threshold)
        #print(y.size, y_l.size)
        if x_l.size == 0 or x_r.size == 0:
            self.tree[node_id] = (self.LEAF_TYPE,np.bincount(y).argmax(),np.bincount(y).astype(float) / y.size)
        else:
            self.tree[node_id] = (self.NON_LEAF_TYPE, best_threshold_id, best_threshold)
            self.__fit_node(x_r, y_r, node_id * 2 + 2, depth + 1)
            self.__fit_node(x_l, y_l,node_id * 2 + 1, depth + 1)
        if self.G_function == self.__entropy:
            g = self.__entrophy_calc(y)
            g_l = self.__entrophy_calc(y_l)
            g_r = self.__entrophy_calc(y_r)
            self.feature_importances_[best_threshold_id] += self.__fimp_calc(y, y_l, y_r, g, g_l, g_r)   
        elif self.G_function == self.__misclass:
            g = self.__g_misclass_calc(y)
            g_l = self.__g_misclass_calc(y_l)
            g_r = self.__g_misclass_calc(y_r)
            self.feature_importances_[best_threshold_id] += self.__fimp_calc(y, y_l, y_r, g, g_l, g_r)
        elif self.G_function == self.__gini:
            g = self.__gini_calc(y)
            g_l = self.__gini_calc(y_l)
            g_r = self.__gini_calc(y_r)
            self.feature_importances_[best_threshold_id] += self.__fimp_calc(y, y_l, y_r, g, g_l, g_r)  

    def __entrophy_calc(self, y):
        return -(np.sum((np.unique(y, return_counts=True)[1] / y.size) * np.log2(np.unique(y, return_counts=True)[1] / y.size)))
        
    def __gini_calc(self, y_a):
        return (1 - np.sum(np.unique(y_a, return_counts=True)[1]** 2 / (y_a.size *y_a.size)))
                
    def __g_misclass_calc(self, y_a):
        return (1 - np.max(np.sum(np.unique(y_a, return_counts=True)[1] / y_a.size)))
    
    def __fimp_calc(self, y, y_l, y_r, g, g_l, g_r):
        return (y.size*g - y_l.size*g_l - y_r.size*g_r)
    
    def fit(self, x, y):
        self.num_class = np.unique(y).size
        
        if(self.feature_importances_ is None):
            self.feature_importances_ = np.zeros(x.shape[1], dtype=float)
        self.__fit_node(x, y, 0, 0)
        self.feature_importances_ /= y.size

    def __predict_class(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_class(x, 2 * node_id + 1)
            else:
                return self.__predict_class(x, 2 * node_id + 2)
        else:
            return node[1]

    def __predict_probs(self, x, node_id):
        node = self.tree[node_id]
        if node[0] == self.__class__.NON_LEAF_TYPE:
            _, feature_id, threshold = node
            if x[feature_id] > threshold:
                return self.__predict_probs(x, 2 * node_id + 1)
            else:
                return self.__predict_probs(x, 2 * node_id + 2)
        else:
            return node[2]
        
    def predict(self, X):
        return np.array([self.__predict_class(x, 0) for x in X])
    
    def predict_probs(self, X):
        return np.array([self.__predict_probs(x, 0) for x in X])

    def fit_predict(self, x_train, y_train, predicted_x):
        self.fit(x_train, y_train)
        return self.predict(predicted_x)

In [3]:
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

In [4]:
wine = load_wine()
X_train, X_test, y_train, y_test = train_test_split(wine.data, wine.target, test_size=0.1, stratify=wine.target)

## Проверка скорости работы на wine

In [5]:
%time clf.fit(X_train, y_train)

CPU times: user 0 ns, sys: 4 ms, total: 4 ms
Wall time: 2.83 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [6]:
%time my_clf.fit(X_train, y_train)

CPU times: user 56 ms, sys: 0 ns, total: 56 ms
Wall time: 54.1 ms


## Проверка качества работы на wine

In [7]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.9440559440559441

In [8]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.9440559440559441

## Подготовка данных Speed Dating Data 

In [9]:
df = pd.read_csv('Speed Dating Data.csv', encoding='cp1251')

In [19]:
for col in df.columns:
    
    if not (type(df[col][0])==np.int64 or\
        type(df[col][0])==np.int64 or\
        type(df[col][0])==np.int or\
        type(df[col][0])==np.float or\
        df[col].isnull().sum()>500) or\
        str(col)=="undergra":
        df.drop(col,axis=1, inplace=True)
df=df.fillna(-100500)
#fix 123,456.00 numbers
def conv(i):
    return np.float(i.replace(",", ""))
a = df.drop('gender', axis=1).iloc[:,:].values.astype(str)
a = np.vectorize(conv)(a)
X = a.astype(np.float)
y = df.loc[:, 'gender'].values.astype(np.int)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)
my_clf = MyDecisionTreeClassifier(min_samples_split=2)
clf = DecisionTreeClassifier(min_samples_split=2)

## Проверка скорости работы на Speed Dating Data 

In [20]:
%time clf.fit(X_train, y_train)


CPU times: user 36 ms, sys: 0 ns, total: 36 ms
Wall time: 34.8 ms


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [21]:
%time my_clf.fit(X_train, y_train)


CPU times: user 232 ms, sys: 0 ns, total: 232 ms
Wall time: 232 ms


## Проверка качества работы на Speed Dating Data

In [22]:
f1_score(y_pred=my_clf.predict(X_test), y_true=y_test, average='macro')

0.9776134853372336

In [23]:
f1_score(y_pred=clf.predict(X_test), y_true=y_test, average='macro')

0.9994033412887827

## Задание 3

In [24]:
#see task 4

## Задание 4

In [25]:
print('DecisionTreeClassifier')
print(pd.Series(index = df.columns[1:], data = clf.feature_importances_).
      sort_values()[::-1].head(10))
print('\n\nMyDecisionTreeClassifier')
print(pd.Series(index = df.columns[1:], data = my_clf.feature_importances_).
      sort_values()[::-1].head(10))

DecisionTreeClassifier
gender      0.456783
idg         0.231786
round       0.149109
wave        0.100333
tuition     0.052896
condtn      0.006876
position    0.001554
mn_sat      0.000663
dec         0.000000
dec_o       0.000000
dtype: float64


MyDecisionTreeClassifier
gender      0.203979
idg         0.106151
wave        0.083969
round       0.044196
condtn      0.015337
tuition     0.012292
mn_sat      0.001362
dec         0.000000
dec_o       0.000000
samerace    0.000000
dtype: float64


## Задание 5

In [28]:
parametrs = {
    "criterion": ["entropy", "gini"],
    "min_samples_split": np.arange(2, 10).tolist(),
    "max_depth": np.arange(2, 16).tolist(),
    "n_estimators": np.arange(50, 400,20).tolist()
    }
RS = RandomizedSearchCV(RandomForestClassifier(random_state=123), param_distributions=parametrs, n_iter=32, cv=3)
RS.fit(X_train, y_train)
print(RS.best_params_)

{'n_estimators': 110, 'min_samples_split': 4, 'max_depth': 15, 'criterion': 'gini'}
